# Multiple Tables

In [17]:
import sqlite3
conn = sqlite3.connect('./billboard-200.db')
cursor = conn.cursor()

In [18]:
cursor.execute('SELECT name from sqlite_master where type= "table"')
cursor.fetchall()

[('albums',), ('acoustic_features',)]

Let's focus in on the albums table.

In [16]:
cursor.execute('PRAGMA table_info(albums);')
cursor.fetchall()

[(0, 'id', 'integer', 0, None, 1),
 (1, 'date', 'text', 0, None, 0),
 (2, 'artist', 'text', 0, None, 0),
 (3, 'album', 'text', 0, None, 0),
 (4, 'rank', 'text', 0, None, 0),
 (5, 'length', 'integer', 0, None, 0),
 (6, 'track_length', 'real', 0, None, 0)]

Now the columns in the albums table look innocent enough, but trouble could be looming.  

In [20]:
cursor.execute('SELECT * from Albums LIMIT 3;')
cursor.fetchall()

[(1, None, None, None, None, None, None),
 (2, '2019-01-19', 'A Boogie Wit da Hoodie', 'Hoodie SZN', '1', 20, 185233.8),
 (3, '2019-01-19', '21 Savage', 'I Am > I Was', '2', 15, 211050.73333333334)]

For example, we have the artist name listed in the albums table.  But notice that each time we store an album by the same artist, we have to repeat that artist's name.  This is violating our Don't Repeat Yourself rule.  Here, it means that if we ever wanted to change the Artist's name, we would have to change this information in multiple places.

It's best to keep the table reserved for attributes directly about the album.  Notice that all of the other information do fit the bill - they are about the album itself.  Of course, we *do* want to connect the artist to our album.  So how do we do that? 

### Solving our Problem

The best way is to create a separate artists table like so.

`artists`

| id        | name           | 
| ------------- |:-------------:| 
| 1               |Prince | 
| 2      | Paul Simon      | 

Then to connect our `artists` table with our `albums` table, we set up our albums table like so. 

| id| date    | rank| length| track_length | artist_id | album
| --|:-------:| ----|-------| --------------| -------| ------|
144 | '2019-01-19'|'143' | 17 | 256500 | 1| The Very Best Of Prince
17155| '2017-06-03'| '154'| 40| 233901| 1|  '4Ever' 
6269 | '2016-07-16' | '68'| 11 | 202174.0 | 2| 'Stranger To Stranger'  

The important column is the `artist_id` column.  This is referred to as a foreign key, or foreign id, because it refers to a number that is foreign to the current table of albums.  The table it refers to is the `artist` table.

We can describe the relationship as an artist `has many` albums, and an album `has one` artist (at least in this case).  In a `has one` to `has many` relationship, the foreign key always goes on the table that `has one`.  So a album has one artist, and the albums table gets the foreign key.  

This makes sense as if we placed the foreign key on the has many table, here artists, we have to add multiple albums foreign key albums for artist that have multiple albums. 

### Querying our tables

Once we have an idea of how our tables fit together, it's not so tricky to determine how to query our tables.  There are really only two questions we can ask:

1. Given an album, what is the artist
2. Given an artist, what are all of the listed albums

Let's tackle these in turn.  Once again this is our tables.

`artists`

| id        | name           | 
| ------------- |:-------------:| 
| 1               |Prince | 
| 2      | Paul Simon      |


`albums`

| id| date    | rank| length| track_length | artist_id | album
| --|:-------:| ----|-------| --------------| -------| ------|
144 | '2019-01-19'|'143' | 17 | 256500 | 1| The Very Best Of Prince
17155| '2017-06-03'| '154'| 40| 233901| 1|  '4Ever' 
6269 | '2016-07-16' | '68'| 11 | 202174.0 | 2| 'Stranger To Stranger'  

Now let's we want to find the artist who wrote `Stranger to Stranger`.  This is the sql query.

```sql
SELECT artists.* FROM artists JOIN albums on albums.artist_id = artists.id WHERE album = 'Stranger to Stranger'
```

This may seem complex, but it really matches what our brain would do.

If we want to answer the question, who wrote the album `Stranger to Stranger`, we first answer the question, find the album whose name is `stranger to stranger`, so perhaps we begin this with the following.

```sql
SELECT * from artists where album = 'Stranger to Stranger'
```

But unfortunately, information about the album name is in a totally different table than our artists table (which has our artist name).  So we need a way to connect the artists table and the albums table.  The way that to do so, is to `join` the two tables by matching rows where the `albums.artist_id = artists.id`.  When we join two tables, we essentially create one large table in memory, with rows linked by the foreign key.  In this case it would look something like this.

| id| date    | rank| length| track_length | artist_id | album | id        | name           | 
| --|:-------:| ----|-------| --------------| -------| --------|-----------| ---------------| 
144 | '2019-01-19'|'143' | 17 | 256500 | 1| The Very Best Of Prince| 1|     Prince
17155| '2017-06-03'| '154'| 40| 233901| 1|  '4Ever' | 1| Prince| 
6269 | '2016-07-16' | '68'| 11 | 202174.0 | 2| 'Stranger To Stranger' | 2| Paul Simon|

So we can see that we have connected the tables, and linked the rows such that `albums.artist_id = artists.id`.

Because we now have both tables loaded into memory, but only need to return information in the `artists` table, we change our `SELECT * ` to `SELECT artists.*` which only displays columns from the artists table.

### Summary

### Additional Resources

[music story](http://developers.music-story.com/developers/artist)